#### Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

#### Load Data

In [2]:
data = pd.read_csv("fake_job_postings.csv")
data.head(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


#### Preprocess Data
- Tokenize 
- asdf
- asfd

In [3]:
X = data.drop('fraudulent', axis=1)
y = data['fraudulent']

Scaling and Encoding

In [4]:
# Encoding categorical variables
le = LabelEncoder()
oe = OneHotEncoder()

X['employment_type'] = le.fit_transform(X['employment_type'].astype(str))
X['required_experience'] = le.fit_transform(X['required_experience'].astype(str))
X['required_education'] = le.fit_transform(X['required_education'].astype(str))

X = oe.fit_transform(X).toarray()

# Scaling the independent variables
sc = StandardScaler()
X = sc.fit_transform(X)

#### Split Data
- training
- validation
- test

In [5]:
# Split the dataset into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#### Build Model

In [6]:
# Define the model architecture
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8873216   
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6

#### Compile Model

In [7]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.001), 
              metrics=['accuracy'])

c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


#### Add Early Stop

In [8]:
# Set early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', patience=10)

#### Train Model

In [9]:
history = model.fit(X_train, y_train, 
                    batch_size=64, 
                    epochs=100, 
                    validation_data=(X_val, y_val), 
                    callbacks=[early_stop])

Epoch 1/100
179/179 [==============================] - 7s 35ms/step - loss: 0.2739 - accuracy: 0.9309 - val_loss: 0.1329 - val_accuracy: 0.9616
Epoch 2/100
179/179 [==============================] - 6s 35ms/step - loss: 0.0963 - accuracy: 0.9760 - val_loss: 0.2089 - val_accuracy: 0.9165
Epoch 3/100
179/179 [==============================] - 6s 35ms/step - loss: 0.0176 - accuracy: 0.9960 - val_loss: 0.2081 - val_accuracy: 0.9168
Epoch 4/100
179/179 [==============================] - 6s 33ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.2213 - val_accuracy: 0.9098
Epoch 5/100
179/179 [==============================] - 6s 32ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 0.2303 - val_accuracy: 0.9070
Epoch 6/100
179/179 [==============================] - 6s 32ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 0.2447 - val_accuracy: 0.9014
Epoch 7/100
179/179 [==============================] - 6s 32ms/step - loss: 2.8985e-04 - accuracy: 1.0000 - val_loss: 0.2669 - val_accur

#### Evaluate Model

In [10]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=64)
print("Test accuracy:", accuracy)

56/56 [==============================] - 1s 12ms/step - loss: 0.3175 - accuracy: 0.8641
Test accuracy: 0.8640939593315125
